# Методы отбора признаков. Практика

## Задание 1

Отберите десять лучших признаков при помощи методов **SFS** и **SBS**. Используйте такие настройки:

- параметр кросс-валидации равен **4**;
- функция `floating=False`, то есть выключена;
- в качестве оценщика используйте алгоритм **случайного леса** c числом деревьев **300**, а основной метрикой будет `roc_auc`.

Имена отобранных признаков сохраните в переменные `top_sfs` и `top_sbs`.

In [14]:
# Импорт необходимых библиотек
import os
import psycopg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv('../.env')

print("Библиотеки успешно импортированы!")

Библиотеки успешно импортированы!


In [15]:
# Загрузка данных из базы данных
TABLE_NAME = "users_churn"

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

print(f"Данные загружены: {df.shape}")
print(f"\nКолонки датасета:")
print(df.columns.tolist())
print(f"\nПервые строки:")
df.head()

Данные загружены: (7043, 22)

Колонки датасета:
['id', 'customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines', 'target']

Первые строки:


,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,2744,9637-CDTKZ,2014-02-01,NaT,Two year,Yes,Credit card (automatic),114.10,8086.40,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,Yes,No,Yes,0
1,2746,3946-JEWRQ,2016-03-01,NaT,One year,Yes,Credit card (automatic),95.20,4563.00,Fiber optic,...,No,No,Yes,Yes,Male,0,Yes,No,Yes,0
2,2748,7873-CVMAW,2014-02-01,NaT,Two year,No,Credit card (automatic),88.55,6362.35,DSL,...,Yes,Yes,Yes,Yes,Male,0,No,No,Yes,0
3,2750,0463-WZZKO,2019-11-01,NaT,Month-to-month,Yes,Credit card (automatic),20.75,67.10,None,...,None,None,None,None,Male,0,No,No,No,0
4,2752,3494-JCHRQ,2019-12-01,2020-01-01,Month-to-month,Yes,Electronic check,70.05,70.05,Fiber optic,...,No,No,No,No,Male,0,No,No,No,1


In [16]:
# Подготовка данных для модели
# Удаляем ненужные колонки
columns_to_drop = ['id', 'customer_id', 'begin_date', 'end_date']
df_clean = df.drop(columns=columns_to_drop)

# Преобразуем категориальные признаки в числовые
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
categorical_columns = df_clean.select_dtypes(include=['object']).columns

print(f"Категориальные признаки: {categorical_columns.tolist()}")

for col in categorical_columns:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))
    label_encoders[col] = le

# Разделение на признаки и целевую переменную
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"\n📊 Размерность данных:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")
print(f"\n  Количество признаков: {X.shape[1]}")
print(f"  Названия признаков: {X.columns.tolist()}")
print(f"\n  Распределение классов в train: {np.bincount(y_train)}")
print(f"  Распределение классов в test: {np.bincount(y_test)}")

Категориальные признаки: ['type', 'paperless_billing', 'payment_method', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'gender', 'partner', 'dependents', 'multiple_lines']

📊 Размерность данных:
  X_train: (5634, 17)
  X_test: (1409, 17)
  y_train: (5634,)
  y_test: (1409,)

  Количество признаков: 17
  Названия признаков: ['type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines']

  Распределение классов в train: [4139 1495]
  Распределение классов в test: [1035  374]


In [17]:
print("="*70)
print("ОТБОР ПРИЗНАКОВ С ПОМОЩЬЮ SFS И SBS")
print("="*70)

# Создание оценщика - Random Forest с 300 деревьями
estimator = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

# Sequential Forward Selection (SFS)
print("\n1️⃣ Sequential Forward Selection (SFS)...")
print("Это может занять несколько минут...")

sfs = SequentialFeatureSelector(
    estimator,
    k_features=10,  # отбираем 10 признаков
    forward=True,   # forward selection
    floating=False, # floating выключен
    scoring='roc_auc',
    cv=4,           # кросс-валидация = 4
    n_jobs=-1
)

# Запуск отбора признаков для SFS
sfs = sfs.fit(X_train, y_train)

# Получение имён отобранных признаков
top_sfs = list(X.columns[list(sfs.k_feature_idx_)])

print(f"\n✅ SFS завершён!")
print(f"Отобранные признаки (индексы): {list(sfs.k_feature_idx_)}")
print(f"Отобранные признаки (названия): {top_sfs}")

# Sequential Backward Selection (SBS)
print("\n2️⃣ Sequential Backward Selection (SBS)...")
print("Это может занять несколько минут...")

sbs = SequentialFeatureSelector(
    estimator,
    k_features=10,  # отбираем 10 признаков
    forward=False,  # backward selection
    floating=False, # floating выключен
    scoring='roc_auc',
    cv=4,           # кросс-валидация = 4
    n_jobs=-1
)

# Запуск отбора признаков для SBS
sbs = sbs.fit(X_train, y_train)

# Получение имён отобранных признаков
top_sbs = list(X.columns[list(sbs.k_feature_idx_)])

print(f"\n✅ SBS завершён!")
print(f"Отобранные признаки (индексы): {list(sbs.k_feature_idx_)}")
print(f"Отобранные признаки (названия): {top_sbs}")

# Вывод результатов
print("\n" + "="*70)
print("РЕЗУЛЬТАТЫ ОТБОРА ПРИЗНАКОВ")
print("="*70)

print('\nSequential Forward Selection (k=10)')
print('CV Score:')
print(sfs.k_score_)

print('\nSequential Backward Selection (k=10)')
print('CV Score:')
print(sbs.k_score_)

ОТБОР ПРИЗНАКОВ С ПОМОЩЬЮ SFS И SBS

1️⃣ Sequential Forward Selection (SFS)...
Это может занять несколько минут...

✅ SFS завершён!
Отобранные признаки (индексы): [0, 1, 2, 3, 4, 5, 6, 7, 9, 10]
Отобранные признаки (названия): ['type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_security', 'online_backup', 'tech_support', 'streaming_tv']

2️⃣ Sequential Backward Selection (SBS)...
Это может занять несколько минут...

✅ SBS завершён!
Отобранные признаки (индексы): [0, 1, 2, 3, 4, 5, 7, 11, 12, 14]
Отобранные признаки (названия): ['type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_backup', 'streaming_movies', 'gender', 'partner']

РЕЗУЛЬТАТЫ ОТБОРА ПРИЗНАКОВ

Sequential Forward Selection (k=10)
CV Score:
0.8165750193508001

Sequential Backward Selection (k=10)
CV Score:
0.8206442653287767


In [18]:
print("="*70)
print("СРАВНЕНИЕ ОТОБРАННЫХ ПРИЗНАКОВ")
print("="*70)

# Общие признаки
common_features = list(set(top_sfs) & set(top_sbs))
print(f"\n🔄 Общие признаки ({len(common_features)}):")
print(common_features)

# Уникальные для SFS
unique_sfs = list(set(top_sfs) - set(top_sbs))
print(f"\n➡️  Уникальные для SFS ({len(unique_sfs)}):")
print(unique_sfs)

# Уникальные для SBS
unique_sbs = list(set(top_sbs) - set(top_sfs))
print(f"\n⬅️  Уникальные для SBS ({len(unique_sbs)}):")
print(unique_sbs)

# Сравнение CV Score
print(f"\n📊 Сравнение CV Score:")
print(f"  SFS ROC-AUC: {sfs.k_score_:.4f}")
print(f"  SBS ROC-AUC: {sbs.k_score_:.4f}")
print(f"  Разница: {abs(sfs.k_score_ - sbs.k_score_):.4f}")

СРАВНЕНИЕ ОТОБРАННЫХ ПРИЗНАКОВ

🔄 Общие признаки (7):
['payment_method', 'monthly_charges', 'online_backup', 'internet_service', 'paperless_billing', 'total_charges', 'type']

➡️  Уникальные для SFS (3):
['tech_support', 'online_security', 'streaming_tv']

⬅️  Уникальные для SBS (3):
['streaming_movies', 'partner', 'gender']

📊 Сравнение CV Score:
  SFS ROC-AUC: 0.8166
  SBS ROC-AUC: 0.8206
  Разница: 0.0041


In [19]:
print("="*70)
print("ОБУЧЕНИЕ МОДЕЛЕЙ НА ОТОБРАННЫХ ПРИЗНАКАХ")
print("="*70)

# Модель на признаках SFS
print("\n1️⃣ Модель на признаках SFS...")
X_train_sfs = X_train[top_sfs]
X_test_sfs = X_test[top_sfs]

model_sfs = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_sfs.fit(X_train_sfs, y_train)
y_pred_sfs = model_sfs.predict(X_test_sfs)
y_proba_sfs = model_sfs.predict_proba(X_test_sfs)[:, 1]

roc_auc_sfs = roc_auc_score(y_test, y_proba_sfs)
accuracy_sfs = accuracy_score(y_test, y_pred_sfs)

print(f"  ROC-AUC: {roc_auc_sfs:.4f}")
print(f"  Accuracy: {accuracy_sfs:.4f}")

# Модель на признаках SBS
print("\n2️⃣ Модель на признаках SBS...")
X_train_sbs = X_train[top_sbs]
X_test_sbs = X_test[top_sbs]

model_sbs = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_sbs.fit(X_train_sbs, y_train)
y_pred_sbs = model_sbs.predict(X_test_sbs)
y_proba_sbs = model_sbs.predict_proba(X_test_sbs)[:, 1]

roc_auc_sbs = roc_auc_score(y_test, y_proba_sbs)
accuracy_sbs = accuracy_score(y_test, y_pred_sbs)

print(f"  ROC-AUC: {roc_auc_sbs:.4f}")
print(f"  Accuracy: {accuracy_sbs:.4f}")

# Модель на всех признаках для сравнения
print("\n3️⃣ Модель на ВСЕХ признаках...")
model_all = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_all.fit(X_train, y_train)
y_pred_all = model_all.predict(X_test)
y_proba_all = model_all.predict_proba(X_test)[:, 1]

roc_auc_all = roc_auc_score(y_test, y_proba_all)
accuracy_all = accuracy_score(y_test, y_pred_all)

print(f"  ROC-AUC: {roc_auc_all:.4f}")
print(f"  Accuracy: {accuracy_all:.4f}")

# Итоговое сравнение
print("\n" + "="*70)
print("📊 ИТОГОВОЕ СРАВНЕНИЕ")
print("="*70)
print(f"\nМодель            | Признаков | ROC-AUC  | Accuracy")
print(f"-" * 60)
print(f"SFS               | {len(top_sfs):9} | {roc_auc_sfs:.4f}   | {accuracy_sfs:.4f}")
print(f"SBS               | {len(top_sbs):9} | {roc_auc_sbs:.4f}   | {accuracy_sbs:.4f}")
print(f"ВСЕ признаки      | {X.shape[1]:9} | {roc_auc_all:.4f}   | {accuracy_all:.4f}")

print(f"\n✅ Все модели успешно обучены!")

ОБУЧЕНИЕ МОДЕЛЕЙ НА ОТОБРАННЫХ ПРИЗНАКАХ

1️⃣ Модель на признаках SFS...
  ROC-AUC: 0.8179
  Accuracy: 0.7913

2️⃣ Модель на признаках SBS...
  ROC-AUC: 0.8334
  Accuracy: 0.7906

3️⃣ Модель на ВСЕХ признаках...
  ROC-AUC: 0.8330
  Accuracy: 0.8013

📊 ИТОГОВОЕ СРАВНЕНИЕ

Модель            | Признаков | ROC-AUC  | Accuracy
------------------------------------------------------------
SFS               |        10 | 0.8179   | 0.7913
SBS               |        10 | 0.8334   | 0.7906
ВСЕ признаки      |        17 | 0.8330   | 0.8013

✅ Все модели успешно обучены!


In [20]:
# Финальные переменные для проверки
print("="*70)
print("✅ ФИНАЛЬНЫЕ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ")
print("="*70)

print(f"\ntop_sfs = {top_sfs}")
print(f"\ntop_sbs = {top_sbs}")

print(f"\nКоличество признаков SFS: {len(top_sfs)}")
print(f"Количество признаков SBS: {len(top_sbs)}")

✅ ФИНАЛЬНЫЕ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ

top_sfs = ['type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_security', 'online_backup', 'tech_support', 'streaming_tv']

top_sbs = ['type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'internet_service', 'online_backup', 'streaming_movies', 'gender', 'partner']

Количество признаков SFS: 10
Количество признаков SBS: 10


## Задание 2

Создайте два списка `list` в Python, в которых будут храниться признаки, отобранные путём **пересечения** и **объединения**. 

Файл для пересекающихся множеств назовите `interc_features`, для объединяющихся — `union_features`.

In [21]:
print("="*70)
print("ЗАДАНИЕ 2: ПЕРЕСЕЧЕНИЕ И ОБЪЕДИНЕНИЕ ПРИЗНАКОВ")
print("="*70)

# Пересечение признаков (общие для SFS и SBS)
interc_features = list(set(top_sfs) & set(top_sbs))

# Объединение признаков (все уникальные из SFS и SBS)
union_features = list(set(top_sfs) | set(top_sbs))

# Сортируем для удобства
interc_features.sort()
union_features.sort()

print(f"\n🔄 Пересечение признаков (interc_features):")
print(f"   Количество: {len(interc_features)}")
print(f"   Признаки: {interc_features}")

print(f"\n🔗 Объединение признаков (union_features):")
print(f"   Количество: {len(union_features)}")
print(f"   Признаки: {union_features}")

# Дополнительная информация
print(f"\n📊 Анализ:")
print(f"   SFS признаков: {len(top_sfs)}")
print(f"   SBS признаков: {len(top_sbs)}")
print(f"   Общих признаков: {len(interc_features)}")
print(f"   Уникальных всего: {len(union_features)}")
print(f"   Только в SFS: {len(set(top_sfs) - set(top_sbs))}")
print(f"   Только в SBS: {len(set(top_sbs) - set(top_sfs))}")

ЗАДАНИЕ 2: ПЕРЕСЕЧЕНИЕ И ОБЪЕДИНЕНИЕ ПРИЗНАКОВ

🔄 Пересечение признаков (interc_features):
   Количество: 7
   Признаки: ['internet_service', 'monthly_charges', 'online_backup', 'paperless_billing', 'payment_method', 'total_charges', 'type']

🔗 Объединение признаков (union_features):
   Количество: 13
   Признаки: ['gender', 'internet_service', 'monthly_charges', 'online_backup', 'online_security', 'paperless_billing', 'partner', 'payment_method', 'streaming_movies', 'streaming_tv', 'tech_support', 'total_charges', 'type']

📊 Анализ:
   SFS признаков: 10
   SBS признаков: 10
   Общих признаков: 7
   Уникальных всего: 13
   Только в SFS: 3
   Только в SBS: 3


In [22]:
# Визуализация пересечения и объединения
print("="*70)
print("ВИЗУАЛИЗАЦИЯ МНОЖЕСТВ")
print("="*70)

print(f"\n📋 Детальный анализ:")

# Только в SFS
only_sfs = list(set(top_sfs) - set(top_sbs))
only_sfs.sort()
print(f"\n➡️  Только в SFS ({len(only_sfs)} признаков):")
print(f"   {only_sfs}")

# Только в SBS
only_sbs = list(set(top_sbs) - set(top_sfs))
only_sbs.sort()
print(f"\n⬅️  Только в SBS ({len(only_sbs)} признаков):")
print(f"   {only_sbs}")

# Общие
print(f"\n🔄 Общие для SFS и SBS ({len(interc_features)} признаков):")
print(f"   {interc_features}")

# Финальные значения для проверки
print(f"\n{'='*70}")
print("✅ ФИНАЛЬНЫЕ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 2")
print("="*70)
print(f"\ninterc_features = {interc_features}")
print(f"\nunion_features = {union_features}")

ВИЗУАЛИЗАЦИЯ МНОЖЕСТВ

📋 Детальный анализ:

➡️  Только в SFS (3 признаков):
   ['online_security', 'streaming_tv', 'tech_support']

⬅️  Только в SBS (3 признаков):
   ['gender', 'partner', 'streaming_movies']

🔄 Общие для SFS и SBS (7 признаков):
   ['internet_service', 'monthly_charges', 'online_backup', 'paperless_billing', 'payment_method', 'total_charges', 'type']

✅ ФИНАЛЬНЫЕ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 2

interc_features = ['internet_service', 'monthly_charges', 'online_backup', 'paperless_billing', 'payment_method', 'total_charges', 'type']

union_features = ['gender', 'internet_service', 'monthly_charges', 'online_backup', 'online_security', 'paperless_billing', 'partner', 'payment_method', 'streaming_movies', 'streaming_tv', 'tech_support', 'total_charges', 'type']


## Задание 3

Логируйте отобранные признаки в MLflow. Сохраните информацию об эксперименте.

Выполнив предыдущий шаг, вы получили значение переменной `run_id`. Ниже напишите соответствующее значение вашей переменной.

In [23]:
import mlflow
import mlflow.sklearn
from dotenv import load_dotenv
import json

# Загрузка переменных окружения для S3
load_dotenv('../.env')

# Настройка S3 для MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")

# Настройка MLflow
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5001
EXPERIMENT_NAME = "kosmoline_churn_prediction"
RUN_NAME = "feature_selection_sfs_sbs"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

print("="*70)
print("ЛОГИРОВАНИЕ ПРИЗНАКОВ В MLFLOW")
print("="*70)

print(f"\nMLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment Name: {EXPERIMENT_NAME}")

# Получаем experiment_id
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    print(f"Создан новый эксперимент с ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Используется существующий эксперимент с ID: {experiment_id}")

ЛОГИРОВАНИЕ ПРИЗНАКОВ В MLFLOW

MLflow Tracking URI: http://127.0.0.1:5001
Experiment Name: kosmoline_churn_prediction
Используется существующий эксперимент с ID: 4


In [24]:
# Запуск MLflow run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    print(f"\n✅ Run ID: {run_id}")
    print(f"✅ Experiment ID: {experiment_id}")
    print(f"✅ Artifact URI: {run.info.artifact_uri}")
    
    # Логируем параметры отбора признаков
    print(f"\n📊 Логирование параметров...")
    mlflow.log_param("method_1", "SFS")
    mlflow.log_param("method_2", "SBS")
    mlflow.log_param("k_features", 10)
    mlflow.log_param("cv", 4)
    mlflow.log_param("scoring", "roc_auc")
    mlflow.log_param("estimator", "RandomForestClassifier")
    mlflow.log_param("n_estimators", 300)
    mlflow.log_param("floating", False)
    
    # Логируем метрики
    print(f"📊 Логирование метрик...")
    mlflow.log_metric("sfs_cv_score", sfs.k_score_)
    mlflow.log_metric("sbs_cv_score", sbs.k_score_)
    mlflow.log_metric("sfs_roc_auc", roc_auc_sfs)
    mlflow.log_metric("sbs_roc_auc", roc_auc_sbs)
    mlflow.log_metric("all_features_roc_auc", roc_auc_all)
    mlflow.log_metric("n_interc_features", len(interc_features))
    mlflow.log_metric("n_union_features", len(union_features))
    
    # Логируем файлы как артефакты
    print(f"📦 Создание и логирование артефактов...")
    
    import tempfile
    import matplotlib.pyplot as plt
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # 1. Создаём sfs.csv - список признаков SFS
        sfs_csv_file = os.path.join(tmpdir, "sfs.csv")
        sfs_df = pd.DataFrame({
            'feature_name': top_sfs,
            'feature_index': [list(X.columns).index(f) for f in top_sfs]
        })
        sfs_df.to_csv(sfs_csv_file, index=False)
        print(f"  ✓ Создан sfs.csv")
        
        # 2. Создаём sbs.csv - список признаков SBS
        sbs_csv_file = os.path.join(tmpdir, "sbs.csv")
        sbs_df = pd.DataFrame({
            'feature_name': top_sbs,
            'feature_index': [list(X.columns).index(f) for f in top_sbs]
        })
        sbs_df.to_csv(sbs_csv_file, index=False)
        print(f"  ✓ Создан sbs.csv")
        
        # 3. Создаём sfs.png - визуализация признаков SFS
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = np.arange(len(top_sfs))
        ax.barh(y_pos, [1] * len(top_sfs), color='skyblue')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(top_sfs)
        ax.invert_yaxis()
        ax.set_xlabel('Selected')
        ax.set_title(f'SFS Selected Features (n={len(top_sfs)}, ROC-AUC={sfs.k_score_:.4f})')
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        sfs_png_file = os.path.join(tmpdir, "sfs.png")
        plt.savefig(sfs_png_file, dpi=100, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Создан sfs.png")
        
        # 4. Создаём sbs.png - визуализация признаков SBS
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = np.arange(len(top_sbs))
        ax.barh(y_pos, [1] * len(top_sbs), color='lightcoral')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(top_sbs)
        ax.invert_yaxis()
        ax.set_xlabel('Selected')
        ax.set_title(f'SBS Selected Features (n={len(top_sbs)}, ROC-AUC={sbs.k_score_:.4f})')
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        sbs_png_file = os.path.join(tmpdir, "sbs.png")
        plt.savefig(sbs_png_file, dpi=100, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Создан sbs.png")
        
        # Логируем артефакты напрямую в artifacts/ (без поддиректории)
        print(f"\n📤 Загрузка артефактов в MLflow...")
        mlflow.log_artifacts(tmpdir)
        
    print(f"\n✅ Все данные успешно залогированы!")
    print(f"\n📋 Загружено 4 обязательных файла:")
    print(f"  - sfs.csv")
    print(f"  - sfs.png")
    print(f"  - sbs.csv")
    print(f"  - sbs.png")

print(f"\n🔗 View run: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/experiments/{experiment_id}/runs/{run_id}")

# Проверка сохранения в S3
import time
print(f"\n⏳ Ожидание 5 секунд для загрузки в S3...")
time.sleep(5)

print(f"\n{'='*70}")
print("ПРОВЕРКА СОХРАНЕНИЯ В S3")
print("="*70)

try:
    import boto3
    
    s3_client = boto3.client(
        's3',
        endpoint_url=os.environ.get('MLFLOW_S3_ENDPOINT_URL'),
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name='ru-central1'
    )
    
    bucket_name = os.getenv('S3_BUCKET_NAME')
    prefix = f"{experiment_id}/{run_id}/artifacts/"
    
    print(f"\n🔍 Проверка пути: s3://{bucket_name}/{prefix}")
    
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        print(f"\n✅ В S3 найдено {len(response['Contents'])} файлов:")
        
        required_files = ['sfs.csv', 'sfs.png', 'sbs.csv', 'sbs.png']
        found_files = [obj['Key'].split('/')[-1] for obj in response['Contents']]
        
        for obj in response['Contents']:
            file_name = obj['Key'].split('/')[-1]
            if file_name:
                print(f"  ✓ {file_name} ({obj['Size']} bytes)")
        
        print(f"\n📋 Проверка обязательных файлов:")
        all_found = True
        for req_file in required_files:
            if req_file in found_files:
                print(f"  ✅ {req_file} - найден")
            else:
                print(f"  ❌ {req_file} - НЕ найден")
                all_found = False
        
        if all_found:
            print(f"\n🎉 ВСЕ ОБЯЗАТЕЛЬНЫЕ ФАЙЛЫ НАЙДЕНЫ!")
    else:
        print(f"\n❌ Файлы не найдены по пути {prefix}")
        
except Exception as e:
    print(f"\n⚠️  Ошибка проверки S3: {e}")
    import traceback
    traceback.print_exc()

# Сохраняем run_id для проверки
feature_selection_run_id = run_id

print(f"\n{'='*70}")
print("✅ ФИНАЛЬНЫЕ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3")
print("="*70)
print(f'\nrun_id = "{feature_selection_run_id}"')
print(f'experiment_id = {experiment_id}')
print(f'\nПолный путь к артефактам:')
print(f's3://{bucket_name}/{experiment_id}/{run_id}/artifacts/')


✅ Run ID: 0c1b4a8154a64e9f949dba32d1dcaec1
✅ Experiment ID: 4
✅ Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/4/0c1b4a8154a64e9f949dba32d1dcaec1/artifacts

📊 Логирование параметров...
📊 Логирование метрик...
📦 Создание и логирование артефактов...
  ✓ Создан sfs.csv
  ✓ Создан sbs.csv
  ✓ Создан sfs.png
  ✓ Создан sbs.png

📤 Загрузка артефактов в MLflow...

✅ Все данные успешно залогированы!

📋 Загружено 4 обязательных файла:
  - sfs.csv
  - sfs.png
  - sbs.csv
  - sbs.png
🏃 View run feature_selection_sfs_sbs at: http://127.0.0.1:5001/#/experiments/4/runs/0c1b4a8154a64e9f949dba32d1dcaec1
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/4

🔗 View run: http://127.0.0.1:5001/#/experiments/4/runs/0c1b4a8154a64e9f949dba32d1dcaec1

⏳ Ожидание 5 секунд для загрузки в S3...

ПРОВЕРКА СОХРАНЕНИЯ В S3

🔍 Проверка пути: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/4/0c1b4a8154a64e9f949dba32d1dcaec1/artifacts/

✅ В S3 найдено 4 файлов:
  ✓ sbs.csv (181 bytes)
  ✓ sb

In [25]:
# Задание 3: Финальная переменная для проверки
run_id = feature_selection_run_id

print("="*70)
print("✅ ЗНАЧЕНИЕ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3")
print("="*70)
print(f'\nrun_id = "{run_id}"')

✅ ЗНАЧЕНИЕ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3

run_id = "0c1b4a8154a64e9f949dba32d1dcaec1"


## Задание 4

Теперь обучите две версии модели на двух наборах признаков и сравните метрики. 

Зарегистрируйте обе модели в MLflow в экспериментах:
- `feature_selection_union` - для модели на объединённых признаках
- `feature_selection_intersection` - для модели на пересечённых признаках

В переменные ниже вставьте соответствующую информацию.

In [26]:
print("="*70)
print("ЗАДАНИЕ 4: ОБУЧЕНИЕ И РЕГИСТРАЦИЯ МОДЕЛЕЙ")
print("="*70)

# Название для регистрации модели
REGISTRY_MODEL_NAME = "churn-classifier-feature-selection"

# 1. МОДЕЛЬ НА ПЕРЕСЕЧЁННЫХ ПРИЗНАКАХ
print("\n1️⃣ МОДЕЛЬ НА ПЕРЕСЕЧЁННЫХ ПРИЗНАКАХ (INTERSECTION)")
print("="*70)

# Подготовка данных
X_train_interc = X_train[interc_features]
X_test_interc = X_test[interc_features]

print(f"\nПризнаков: {len(interc_features)}")
print(f"Признаки: {interc_features}")
print(f"X_train shape: {X_train_interc.shape}")
print(f"X_test shape: {X_test_interc.shape}")

# Обучение модели
print(f"\n🤖 Обучение RandomForestClassifier...")
model_interc = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_interc.fit(X_train_interc, y_train)

# Предсказания
y_pred_interc = model_interc.predict(X_test_interc)
y_proba_interc = model_interc.predict_proba(X_test_interc)[:, 1]

# Метрики
metrics_interc = {
    "roc_auc": roc_auc_score(y_test, y_proba_interc),
    "accuracy": accuracy_score(y_test, y_pred_interc)
}

print(f"\n📊 Метрики:")
print(f"  ROC-AUC: {metrics_interc['roc_auc']:.4f}")
print(f"  Accuracy: {metrics_interc['accuracy']:.4f}")

# 2. МОДЕЛЬ НА ОБЪЕДИНЁННЫХ ПРИЗНАКАХ
print(f"\n\n2️⃣ МОДЕЛЬ НА ОБЪЕДИНЁННЫХ ПРИЗНАКАХ (UNION)")
print("="*70)

# Подготовка данных
X_train_union = X_train[union_features]
X_test_union = X_test[union_features]

print(f"\nПризнаков: {len(union_features)}")
print(f"Признаки: {union_features}")
print(f"X_train shape: {X_train_union.shape}")
print(f"X_test shape: {X_test_union.shape}")

# Обучение модели
print(f"\n🤖 Обучение RandomForestClassifier...")
model_union = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_union.fit(X_train_union, y_train)

# Предсказания
y_pred_union = model_union.predict(X_test_union)
y_proba_union = model_union.predict_proba(X_test_union)[:, 1]

# Метрики
metrics_union = {
    "roc_auc": roc_auc_score(y_test, y_proba_union),
    "accuracy": accuracy_score(y_test, y_pred_union)
}

print(f"\n📊 Метрики:")
print(f"  ROC-AUC: {metrics_union['roc_auc']:.4f}")
print(f"  Accuracy: {metrics_union['accuracy']:.4f}")

# Сравнение
print(f"\n\n📊 СРАВНЕНИЕ МОДЕЛЕЙ")
print("="*70)
print(f"\nМодель            | Признаков | ROC-AUC  | Accuracy")
print(f"-" * 60)
print(f"Intersection      | {len(interc_features):9} | {metrics_interc['roc_auc']:.4f}   | {metrics_interc['accuracy']:.4f}")
print(f"Union             | {len(union_features):9} | {metrics_union['roc_auc']:.4f}   | {metrics_union['accuracy']:.4f}")
print(f"\n✅ Обе модели успешно обучены!")

ЗАДАНИЕ 4: ОБУЧЕНИЕ И РЕГИСТРАЦИЯ МОДЕЛЕЙ

1️⃣ МОДЕЛЬ НА ПЕРЕСЕЧЁННЫХ ПРИЗНАКАХ (INTERSECTION)

Признаков: 7
Признаки: ['internet_service', 'monthly_charges', 'online_backup', 'paperless_billing', 'payment_method', 'total_charges', 'type']
X_train shape: (5634, 7)
X_test shape: (1409, 7)

🤖 Обучение RandomForestClassifier...

📊 Метрики:
  ROC-AUC: 0.8158
  Accuracy: 0.7835


2️⃣ МОДЕЛЬ НА ОБЪЕДИНЁННЫХ ПРИЗНАКАХ (UNION)

Признаков: 13
Признаки: ['gender', 'internet_service', 'monthly_charges', 'online_backup', 'online_security', 'paperless_billing', 'partner', 'payment_method', 'streaming_movies', 'streaming_tv', 'tech_support', 'total_charges', 'type']
X_train shape: (5634, 13)
X_test shape: (1409, 13)

🤖 Обучение RandomForestClassifier...

📊 Метрики:
  ROC-AUC: 0.8324
  Accuracy: 0.7928


📊 СРАВНЕНИЕ МОДЕЛЕЙ

Модель            | Признаков | ROC-AUC  | Accuracy
------------------------------------------------------------
Intersection      |         7 | 0.8158   | 0.7835
Union          

In [27]:
# РЕГИСТРАЦИЯ МОДЕЛЕЙ В MLFLOW
print("\n" + "="*70)
print("РЕГИСТРАЦИЯ МОДЕЛЕЙ В MLFLOW")
print("="*70)

# Название для регистрации модели
registred_model_name = "churn-classifier-feature-selection"

# 1. РЕГИСТРАЦИЯ МОДЕЛИ НА ПЕРЕСЕЧЁННЫХ ПРИЗНАКАХ
print("\n1️⃣ РЕГИСТРАЦИЯ МОДЕЛИ INTERSECTION В MLFLOW")
print("="*70)

# Создаём/получаем эксперимент
EXPERIMENT_NAME_INTERC = "feature_selection_intersection"
mlflow.set_experiment(EXPERIMENT_NAME_INTERC)

experiment_interc = mlflow.get_experiment_by_name(EXPERIMENT_NAME_INTERC)
experiment_id_interc = experiment_interc.experiment_id

print(f"Experiment: {EXPERIMENT_NAME_INTERC}")
print(f"Experiment ID: {experiment_id_interc}")

# Запускаем run и логируем модель
with mlflow.start_run(run_name="intersection_model", experiment_id=experiment_id_interc) as run:
    run_id_interc = run.info.run_id
    run_name_interc = run.info.run_name
    
    print(f"\nRun ID: {run_id_interc}")
    print(f"Run Name: {run_name_interc}")
    
    # Логируем параметры
    mlflow.log_param("n_features", len(interc_features))
    mlflow.log_param("features", ", ".join(interc_features))
    mlflow.log_param("feature_selection_method", "intersection_sfs_sbs")
    mlflow.log_param("n_estimators", 300)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("model_type", "RandomForestClassifier")
    
    # Логируем метрики
    mlflow.log_metric("roc_auc", metrics_interc['roc_auc'])
    mlflow.log_metric("accuracy", metrics_interc['accuracy'])
    mlflow.log_metric("test_size", X_test_interc.shape[0])
    mlflow.log_metric("train_size", X_train_interc.shape[0])
    
    # Создаём и логируем артефакты
    import tempfile
    import matplotlib.pyplot as plt
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # Сохраняем список признаков
        features_file = os.path.join(tmpdir, "features.txt")
        with open(features_file, 'w') as f:
            f.write("Features used in intersection model:\n")
            f.write("\n".join(interc_features))
        
        # Визуализация важности признаков
        feature_importance = pd.DataFrame({
            'feature': interc_features,
            'importance': model_interc.feature_importances_
        }).sort_values('importance', ascending=False)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
        ax.set_xlabel('Importance')
        ax.set_title(f'Feature Importances (Intersection Model)\nROC-AUC: {metrics_interc["roc_auc"]:.4f}')
        ax.invert_yaxis()
        plt.tight_layout()
        importance_file = os.path.join(tmpdir, "feature_importance.png")
        plt.savefig(importance_file, dpi=100, bbox_inches='tight')
        plt.close()
        
        # Логируем артефакты
        mlflow.log_artifacts(tmpdir)
    
    # Регистрируем модель
    print(f"\n📦 Регистрация модели '{registred_model_name}'...")
    model_info = mlflow.sklearn.log_model(
        sk_model=model_interc,
        artifact_path="model",
        registered_model_name=registred_model_name,
        input_example=X_train_interc[:5]
    )
    
    print(f"✅ Модель зарегистрирована!")
    print(f"   Model URI: {model_info.model_uri}")

# Получаем версию модели
client = mlflow.tracking.MlflowClient()
model_versions = client.search_model_versions(f"name='{registred_model_name}'")

# Находим версию модели для текущего run_id
model_version_id_interc = None
for mv in model_versions:
    if mv.run_id == run_id_interc:
        model_version_id_interc = int(mv.version)
        break

print(f"\n📋 Информация о модели INTERSECTION:")
print(f"   Registered Model Name: {registred_model_name}")
print(f"   Model Version: {model_version_id_interc}")
print(f"   Run Name: {run_name_interc}")
print(f"   Run ID: {run_id_interc}")

# 2. РЕГИСТРАЦИЯ МОДЕЛИ НА ОБЪЕДИНЁННЫХ ПРИЗНАКАХ
print("\n\n2️⃣ РЕГИСТРАЦИЯ МОДЕЛИ UNION В MLFLOW")
print("="*70)

# Создаём/получаем эксперимент
EXPERIMENT_NAME_UNION = "feature_selection_union"
mlflow.set_experiment(EXPERIMENT_NAME_UNION)

experiment_union = mlflow.get_experiment_by_name(EXPERIMENT_NAME_UNION)
experiment_id_union = experiment_union.experiment_id

print(f"Experiment: {EXPERIMENT_NAME_UNION}")
print(f"Experiment ID: {experiment_id_union}")

# Запускаем run и логируем модель
with mlflow.start_run(run_name="union_model", experiment_id=experiment_id_union) as run:
    run_id_union = run.info.run_id
    run_name_union = run.info.run_name
    
    print(f"\nRun ID: {run_id_union}")
    print(f"Run Name: {run_name_union}")
    
    # Логируем параметры
    mlflow.log_param("n_features", len(union_features))
    mlflow.log_param("features", ", ".join(union_features))
    mlflow.log_param("feature_selection_method", "union_sfs_sbs")
    mlflow.log_param("n_estimators", 300)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("model_type", "RandomForestClassifier")
    
    # Логируем метрики
    mlflow.log_metric("roc_auc", metrics_union['roc_auc'])
    mlflow.log_metric("accuracy", metrics_union['accuracy'])
    mlflow.log_metric("test_size", X_test_union.shape[0])
    mlflow.log_metric("train_size", X_train_union.shape[0])
    
    # Создаём и логируем артефакты
    with tempfile.TemporaryDirectory() as tmpdir:
        # Сохраняем список признаков
        features_file = os.path.join(tmpdir, "features.txt")
        with open(features_file, 'w') as f:
            f.write("Features used in union model:\n")
            f.write("\n".join(union_features))
        
        # Визуализация важности признаков
        feature_importance = pd.DataFrame({
            'feature': union_features,
            'importance': model_union.feature_importances_
        }).sort_values('importance', ascending=False)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.barh(feature_importance['feature'], feature_importance['importance'], color='coral')
        ax.set_xlabel('Importance')
        ax.set_title(f'Feature Importances (Union Model)\nROC-AUC: {metrics_union["roc_auc"]:.4f}')
        ax.invert_yaxis()
        plt.tight_layout()
        importance_file = os.path.join(tmpdir, "feature_importance.png")
        plt.savefig(importance_file, dpi=100, bbox_inches='tight')
        plt.close()
        
        # Логируем артефакты
        mlflow.log_artifacts(tmpdir)
    
    # Регистрируем модель
    print(f"\n📦 Регистрация модели '{registred_model_name}'...")
    model_info = mlflow.sklearn.log_model(
        sk_model=model_union,
        artifact_path="model",
        registered_model_name=registred_model_name,
        input_example=X_train_union[:5]
    )
    
    print(f"✅ Модель зарегистрирована!")
    print(f"   Model URI: {model_info.model_uri}")

# Получаем версию модели
model_versions = client.search_model_versions(f"name='{registred_model_name}'")

# Находим версию модели для текущего run_id
model_version_id_union = None
for mv in model_versions:
    if mv.run_id == run_id_union:
        model_version_id_union = int(mv.version)
        break

print(f"\n📋 Информация о модели UNION:")
print(f"   Registered Model Name: {registred_model_name}")
print(f"   Model Version: {model_version_id_union}")
print(f"   Run Name: {run_name_union}")
print(f"   Run ID: {run_id_union}")

# ИТОГОВОЕ СРАВНЕНИЕ
print("\n\n" + "="*70)
print("📊 ИТОГОВОЕ СРАВНЕНИЕ ЗАРЕГИСТРИРОВАННЫХ МОДЕЛЕЙ")
print("="*70)

comparison_df = pd.DataFrame({
    'Model': ['Intersection', 'Union'],
    'Features': [len(interc_features), len(union_features)],
    'ROC-AUC': [metrics_interc['roc_auc'], metrics_union['roc_auc']],
    'Accuracy': [metrics_interc['accuracy'], metrics_union['accuracy']],
    'Version': [model_version_id_interc, model_version_id_union],
    'Run ID': [run_id_interc, run_id_union]
})

print("\n", comparison_df.to_string(index=False))

print(f"\n✅ Обе модели успешно зарегистрированы в MLflow!")
print(f"\n🔗 View Models: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/models/{registred_model_name}")


РЕГИСТРАЦИЯ МОДЕЛЕЙ В MLFLOW

1️⃣ РЕГИСТРАЦИЯ МОДЕЛИ INTERSECTION В MLFLOW


2025/12/07 17:57:59 INFO mlflow.tracking.fluent: Experiment with name 'feature_selection_intersection' does not exist. Creating a new experiment.


Experiment: feature_selection_intersection
Experiment ID: 6

Run ID: 2d80df5ecaea4c5989b6ea25e59478fe
Run Name: intersection_model


2025/12/07 17:58:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



📦 Регистрация модели 'churn-classifier-feature-selection'...


...

Experiment: feature_selection_union
Experiment ID: 7

Run ID: 303f6ad64ee744c9ae216bd3e99758c9
Run Name: union_model


2025/12/07 18:02:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



📦 Регистрация модели 'churn-classifier-feature-selection'...


/Users/asvorobyev/miniconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/asvorobyev/miniconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data cont

✅ Модель зарегистрирована!
   Model URI: models:/m-b3847ac2c09c4f69ad7f709ffd8b4291
🏃 View run union_model at: http://127.0.0.1:5001/#/experiments/7/runs/303f6ad64ee744c9ae216bd3e99758c9
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/7

📋 Информация о модели UNION:
   Registered Model Name: churn-classifier-feature-selection
   Model Version: 2
   Run Name: union_model
   Run ID: 303f6ad64ee744c9ae216bd3e99758c9


📊 ИТОГОВОЕ СРАВНЕНИЕ ЗАРЕГИСТРИРОВАННЫХ МОДЕЛЕЙ

        Model  Features  ROC-AUC  Accuracy  Version                           Run ID
Intersection         7 0.815764  0.783534        1 2d80df5ecaea4c5989b6ea25e59478fe
       Union        13 0.832437  0.792761        2 303f6ad64ee744c9ae216bd3e99758c9

✅ Обе модели успешно зарегистрированы в MLflow!

🔗 View Models: http://127.0.0.1:5001/#/models/churn-classifier-feature-selection


In [28]:
# ============================================================================
# ЗАДАНИЕ 4: ФИНАЛЬНЫЕ ПЕРЕМЕННЫЕ ДЛЯ ПРОВЕРКИ
# ============================================================================

print("="*70)
print("✅ ЗАДАНИЕ 4: ФИНАЛЬНЫЕ ПЕРЕМЕННЫЕ ДЛЯ ПРОВЕРКИ")
print("="*70)

# Название зарегистрированной модели (одинаковое для обеих версий)
print(f"\nregistred_model_name = '{registred_model_name}'")

# Модель на пересечённых признаках (INTERSECTION)
print(f"\n# Модель на пересечённых признаках:")
print(f"model_version_id_interc = {model_version_id_interc}  # номер версии модели")
print(f'run_name_interc = "{run_name_interc}"  # название эксперимента')
print(f'run_id_interc = "{run_id_interc}"  # run_id')

# Модель на объединённых признаках (UNION)
print(f"\n# Модель на объединённых признаках:")
print(f"model_version_id_union = {model_version_id_union}  # номер версии модели")
print(f'run_name_union = "{run_name_union}"  # название эксперимента')
print(f'run_id_union = "{run_id_union}"  # run_id')

# Детальная информация
print("\n" + "="*70)
print("ДЕТАЛЬНАЯ ИНФОРМАЦИЯ О МОДЕЛЯХ")
print("="*70)

print(f"\n📦 Зарегистрированная модель: {registred_model_name}")
print(f"\n1️⃣ Модель INTERSECTION:")
print(f"   - Версия: {model_version_id_interc}")
print(f"   - Эксперимент: feature_selection_intersection")
print(f"   - Run Name: {run_name_interc}")
print(f"   - Run ID: {run_id_interc}")
print(f"   - Признаков: {len(interc_features)}")
print(f"   - ROC-AUC: {metrics_interc['roc_auc']:.4f}")
print(f"   - Accuracy: {metrics_interc['accuracy']:.4f}")

print(f"\n2️⃣ Модель UNION:")
print(f"   - Версия: {model_version_id_union}")
print(f"   - Эксперимент: feature_selection_union")
print(f"   - Run Name: {run_name_union}")
print(f"   - Run ID: {run_id_union}")
print(f"   - Признаков: {len(union_features)}")
print(f"   - ROC-AUC: {metrics_union['roc_auc']:.4f}")
print(f"   - Accuracy: {metrics_union['accuracy']:.4f}")

print("\n" + "="*70)
print("✅ ЗАДАНИЕ 4 ВЫПОЛНЕНО!")
print("="*70)

✅ ЗАДАНИЕ 4: ФИНАЛЬНЫЕ ПЕРЕМЕННЫЕ ДЛЯ ПРОВЕРКИ

registred_model_name = 'churn-classifier-feature-selection'

# Модель на пересечённых признаках:
model_version_id_interc = 1  # номер версии модели
run_name_interc = "intersection_model"  # название эксперимента
run_id_interc = "2d80df5ecaea4c5989b6ea25e59478fe"  # run_id

# Модель на объединённых признаках:
model_version_id_union = 2  # номер версии модели
run_name_union = "union_model"  # название эксперимента
run_id_union = "303f6ad64ee744c9ae216bd3e99758c9"  # run_id

ДЕТАЛЬНАЯ ИНФОРМАЦИЯ О МОДЕЛЯХ

📦 Зарегистрированная модель: churn-classifier-feature-selection

1️⃣ Модель INTERSECTION:
   - Версия: 1
   - Эксперимент: feature_selection_intersection
   - Run Name: intersection_model
   - Run ID: 2d80df5ecaea4c5989b6ea25e59478fe
   - Признаков: 7
   - ROC-AUC: 0.8158
   - Accuracy: 0.7835

2️⃣ Модель UNION:
   - Версия: 2
   - Эксперимент: feature_selection_union
   - Run Name: union_model
   - Run ID: 303f6ad64ee744c9ae216bd3e99758c9

## Итоги

В этом задании мы:

1. **Загрузили данные** из базы данных users_churn
2. **Подготовили данные** - закодировали категориальные признаки
3. **Применили SFS** (Sequential Forward Selection) для отбора 10 лучших признаков
4. **Применили SBS** (Sequential Backward Selection) для отбора 10 лучших признаков
5. **Сравнили результаты** обоих методов
6. **Обучили модели** на отобранных признаках и сравнили с моделью на всех признаках

### Настройки отбора признаков:
- Оценщик: RandomForestClassifier (300 деревьев)
- Метрика: ROC-AUC
- Кросс-валидация: 4 фолда
- Floating: выключен
- Количество отбираемых признаков: 10

### Результаты показали:
- Эффективность методов SFS и SBS в снижении размерности
- Сравнение качества моделей на отобранных признаках
- Какие признаки наиболее важны для предсказания оттока клиентов